In [1]:
import csv
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from feature import Feature
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatool.preprocess import  Preprocessor
from datatool.maneger import DataManager

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
path = "../corpus/data_augment.csv"

In [4]:
data = []
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

In [5]:
def make_classes_dict(data):
    classes_dict = dict()
    for row in data:
        c = row[1]
        if c not in classes_dict:
            classes_dict[c] = len(classes_dict)
    return classes_dict

In [6]:
def class_filter(data, remain_classes:list):
    new_data = []
    for row in data:
        c = row[1]
        if c in remain_classes:
            new_data.append(row)
    return new_data

In [7]:
classes_dict_ = make_classes_dict(data)

In [8]:
remain_classes = "how what when where who why YN plain".split()
data_n = class_filter(data, remain_classes)
classes_dict = dict(zip(remain_classes, list(range(len(remain_classes)))))

In [9]:
def extract_X_y(data, classes_dict):
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(classes_dict[d[1]])
    return X, y

In [10]:
import random
def extract_X_y_limit(data, classes_dict:dict, limit=400):
    data_ = random.sample(data, len(data))
    X = []
    y = []
    each_len = dict(zip(classes_dict.keys(), [0]*len(classes_dict)))
    for d in data_:
        if each_len[d[1]] <= limit:
            X.append(d[0])
            y.append(classes_dict[d[1]])
            each_len[d[1]] += 1
    return X, y

In [11]:
# X, y = extract_X_y(data_n, classes_dict)

In [12]:
X, y = extract_X_y_limit(data_n, classes_dict)

In [13]:
len(X)

3208

In [14]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [15]:
F = Feature()
F.set_preprocessor(Preprocessor())
F.make_features(X_train_str)

X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

In [16]:
F_path = "../X_y_data/"
F_name = "typeClassify_F2.pickle"
import dill
import pickle
# featureM = DataManager(F_path, format_="dill")
with open(F_path+F_name, "wb") as f:
    pickle.dump(F, f)

In [17]:
lr = LogisticRegression(solver='sag', max_iter=100)
lr.fit(X_train, y_train)

/home/yamada/.local/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(solver='sag')

In [18]:
y_pred = lr.predict(X_test)

In [19]:
print(classes_dict)
dis = 30
for y_p, x_s in zip(y_pred[:dis], X_test_str[:dis]):
    print("{0} : {1}".format(y_p, x_s))

{'how': 0, 'what': 1, 'when': 2, 'where': 3, 'who': 4, 'why': 5, 'YN': 6, 'plain': 7}
0 : 今度の週末に公園に桃の花を見に行くのはどうだい？
0 : きょうのバスケの練習はどうだったの、ダグ？
6 : 実際、日本で9月1日は防災の日になっています。
7 : 来年は亥年だから、ここにある年賀状にはかわいいイノシシが描かれているんだね。
6 : 何か読むものを買ったの？
6 : コーヒーとケーキが欲しい人？
6 : どんな洋服をスーツケースに入れた？
5 : さつまいもを使ったらどうですか?
5 : ラッキーもぼくたちといっしょに行っていい？
2 : エレベーターの中でないならば、いつどこでこのような売り込みのしかたをするのでしょうか。
2 : サム、いつお母さんにインタビューするつもり？
2 : このお店はいつオープンしましたか？
2 : モニカはいつ、アメリカでダンスを勉強することになっているの？
4 : ぼくへのカード？
4 : それは、だれの桃だったの？
6 : 鯛湖のほとりにあるとても小さな町なの。
6 : 型どおりに答えるべきではありません。
5 : 競馬場でついに大穴を当てたのですか。
6 : シドニーの人口は何人なの？
0 : おふたりはどうやって知り合ったんですか？
3 : どこに着陸したいんだい？
3 : ここからどこへ行くの？
7 : おいでよ。
7 : 日曜日におもしろい映画を見たんだ。
1 : どうしてそんなことを彼にしたのですか？
6 : きみたちには、兄弟と姉妹が何人いるの？
2 : エレベーターの中でないならば、いつどこでこのような売り込みのしかたをするのでしょうか。
1 : 何て書いてあったの？
1 : 何だい？
0 : このウサギちゃんはどう？


In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[ 98  10   0   0   0  11  15   2]
 [ 13  96   0   1   0   3   9   4]
 [  0   1 120   0   0   0   0   0]
 [  0   1   0  95   0   3   0   0]
 [  0   0   0   0 120   1   3   1]
 [  8  13   0   1   0  90  10   8]
 [  5   8   1   2   3  11  76   8]
 [  1   1   0   0   1   4   7  98]]
accuracy =  0.8234683281412254


In [21]:
model_path = "../models/"
model_name = "typeClassify_M2.pickle"
modelM = DataManager(model_path)
print(model_name)

typeClassify_M2.pickle


In [22]:
modelM.save_data(model_name, lr)

success save : ../models/typeClassify_M2.pickle
